In [4]:
# Run this cell once when starting on notebook.
# On Google Colab, wait for the Google Drive permission prompts before proceeding
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
try:
    %load_ext jupyter_ai_magics
except:
    print("%%ai cells will not work in this notebook")
    print("Please use Gemini for AI queries instead")
from datascience import *
import numpy as np
import math

DATA_FOLDER="data/"
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  !mkdir -p /content/data
  !gdown --folder https://drive.google.com/drive/folders/1i8dRvMYzQv2HOBTfTUzjg3vQ42UZzwa3?usp=drive_link -O /content/data
  DATA_FOLDER = DATA_FOLDER.replace("data/", "/content/data/")
  !ls -l /content/data
except:
  print("Google Drive not mounted; this is normal on Jupyter Hub")

import ast
from collections import Counter

%%ai cells will not work in this notebook
Please use Gemini for AI queries instead
Mounted at /content/drive
Retrieving folder contents
Processing file 1V5UMLFvQ2P_6tZw3-5EPVjamB3eLEHaP cir_info.csv
Processing file 1RiOeeIxTq_OrRKeBN5JM42WRkifcQ4AA credits.csv
Processing file 1yuyo7TcrtiLQnpa7YtPp2SMikO3TPlbm movies.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1V5UMLFvQ2P_6tZw3-5EPVjamB3eLEHaP
To: /content/data/cir_info.csv
100% 589/589 [00:00<00:00, 2.09MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RiOeeIxTq_OrRKeBN5JM42WRkifcQ4AA
To: /content/data/credits.csv
100% 174k/174k [00:00<00:00, 26.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yuyo7TcrtiLQnpa7YtPp2SMikO3TPlbm
To: /content/data/movies.csv
100% 525k/525k [00:00<00:00, 65.0MB/s]
Download completed
total 1904
-rw-r--r-- 1 root root     589 May 20 19:32 cir_info.csv
-rw-r--r-- 1 root root 

# CS5A S25 Final Project: Movies

* Please refer to the general instructions in [this document](https://docs.google.com/document/d/1qgS-GPKsbcbqNq8bbDk8kiB0bH-HRZcfN4La75bOtWU/edit?usp=sharing) before starting.
* You may work on either JupyterHub or Google Colab


If working on Colab:

* The Google Colab version is [this folder](https://drive.google.com/drive/folders/1wsVszAufxxmSR7_uoOoR6QBQmjlurPNx?usp=drive_link)
* Make a copy of the notebook in your group folder for the final team project before starting to make edits.
* If/when working in the same file, be sure to coordinate with your group so that only one member of the group is editing at a time; Colab doesn't handle simulataneous editing very well.





## Names

Please list all students that were a member of this team

1. Student Name 1
2. Student Name 2
3. Student Name 3
4. Student Name 4

## Member Responsibilities


*Write your team member responsibility distribution here* (See [instructions](https://docs.google.com/document/d/1qgS-GPKsbcbqNq8bbDk8kiB0bH-HRZcfN4La75bOtWU/edit?tab=t.0#bookmark=id.igbdergm85kj))

## The IMDB Movies Dataset

What can we say about the success of a movie before it is released? Are there certain companies (Pixar?) that have found a consistent formula? Given that major films costing over $100 million to produce can still flop, this question is more important than ever to the industry. Film aficionados might have different interests. Can we predict which films will be highly rated, whether or not they are a commercial success?

This is a great place to start digging in to those questions, with data on the director, budget, rating, genres and revenues of thousands of films.

You will be working with three seperate datasets throughout this final project. You might be required to even join these datasets at some point. Lets go through what each dataset contains:

1. *movies.csv*
- `id`: The iMDB movie ID
- `budget`: The budget in dollars for that film
- `original_language`: The original language the movie was released in
- `original_title`: The original title of the movie
- `popularity`: A custom iMDB score that tracks how popular the movie is amongst its users
- `release_date`: The release date of the movie
- `revenue`: The revenue earned in dollars at the time of release in theatres
- `runtime`: The runtime of the movie in minutes
- `title`: The title of the movie
- `vote_average`: The average vote score given by iMDB users
- `vote_count`: The number of votes a movie has.
- `genres`: An underscore seperated string of genres that the movie comes under.


2. *credits.csv*
- `id`: The iMDB movie ID
- `title`: The movie title
- `Director`: The director of the movie

3. *cir_info.csv*
- `year`: The year
- `CIR`: The Cumulative Inflation Rate from that year to 2023

Please refer to [Chapter 11: Hypothesis testing](https://inferentialthinking.com/chapters/11/Testing_Hypotheses.html) and [Chapter 12.1: A/B Testing](https://inferentialthinking.com/chapters/12/1/AB_Testing.html) when completing this final project as many of the questions use concepts covered there!

**Q1**. The first step to working with data is cleaning and organizing your data. Go through your datasets and make sure you understand all the columns and their data types. Combine the __movie__ and __credits__ tables so that you have one table with all the information you need. Additionally, make sure to perform the following data cleaning steps:
1. Remove any rows with a nan value in it
2. Remove any rows with a revenue AND budget of 0

In [7]:
movies = Table().read_table(DATA_FOLDER + 'movies.csv')
credits = Table().read_table(DATA_FOLDER + 'credits.csv')
movies.show(3)
credits.show(3)

budget,id,original_language,original_title,popularity,release_date,revenue,runtime,title,vote_average,vote_count,genres
237000000,19995,en,Avatar,150.438,2009-12-10,2787965087,162,Avatar,7.2,11800,Action_Adventure_Fantasy_Science Fiction
300000000,285,en,Pirates of the Caribbean: At World's End,139.083,2007-05-19,961000000,169,Pirates of the Caribbean: At World's End,6.9,4500,Adventure_Fantasy_Action
245000000,206647,en,Spectre,107.377,2015-10-26,880674609,148,Spectre,6.3,4466,Action_Adventure_Crime


id,title,Director
19995,Avatar,James Cameron
285,Pirates of the Caribbean: At World's End,Gore Verbinski
206647,Spectre,Sam Mendes


In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Q2.** Before we get started, we need a score for every movie. We could use the `vote_average` of the movie as the score but using this won't be fair enough since a movie with a 8.9 average and only 3 votes cannot be considered better than the movie with 7.8 as the average but with 40 votes.

So we'll be using IMDB's weighted rating (wr) which is given as:

$$ \text{Weighted Rating (WR)} = (\frac{v}{v+m} \times R) + (\frac{m}{v+m} \times C)$$

where,

* *v* is the number of votes for the movie;
* *m* is the minimum votes required to be listed in the chart;
* *R* is the average rating of the movie; And
* *C* is the mean vote across the filtered dataset

We already have *v (vote_count)* and *R (vote_average)*. The next step is to determine an appropriate value for *m*, the minimum votes required to be listed in the chart. We will use 90th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 90% of the movies in the list. Add a new column called `custom_score` which calculates this rating for each movie.

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Q3.** Now lets get into some data analysis! We'll look at Genres first. Maybe directors like to make a particular kind of movie. Plot the top 10 genres with the highest movie count.

**HINT:** You'll have noticed that the genres for each movie are seperated by underscores. A movie can have multiple genres, make sure you consider each one individually by splitting these strings

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

A major indicator of a movie's success is the profit it makes. The profit is dependent on two components, the budget for the film and its revenue in the box office. The next few questions will be looking into these variables and how they affect the data we have.

**Q4.1** First, let's create a new column that calculates the profit for each movie called `profit`, and then also visualize the correlation between the two variables `revenue` and `budget`. Is there a correlation?

**Hint:** $ \text{Profit} = \text{Revenue} - \text{Budget} $

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_



**Q4.2** Lets see how much a movie would make if it was released in 2023 instead of the year it was originally released in. Load *cir_info.csv* from the data folder and **join** it with your dataset.
The CIR is the cumulative inflation rate from that year to 2023. Generate three new columns called `adjusted_budget`, `adjusted_revenue`, and `adjusted_profit` that account for inflation of the US Dollar. Then plot your revenue and budget correlation visualization again.

**Hint:** It is vital to note that the CIR times any value gives you the inflation adjustment and NOT the inflated value itself. So for example, the adjusted budget calculation should be $$ \text{budget} + \text{budget}*\text{CIR}/100 $$

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Q5.1** Sometimes movies with small budgets make big waves in the box office. Lets look at Indie hits! Show the top 10 movies that had their budgets in the bottom 20 percentile but made the biggest profit. (Use adjusted profit and adjusted revenue).

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Q5.2**  Next, let's look at which directors are good at making blockbusters. Identify the 5 most profitable directors and their favorite genres. (Use adjusted profit)

**Hint:** `Counter()` might help you out. Look it up!

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Q6** Looks like our table came with 2 columns, one called `original_title` and another called `title`. It seems weird that we would have two columns like this. As a data scientist, you now have to investigate! What seems to be the main reason for having two seperate columns? You will need to show both code and an analysis.

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Q7.** Nobody likes sitting through long, boring movies. Maybe nobody likes sitting through long movies in general. In this question you will be testing the hypothesis that excessively long movies do not have a good weighted rating due to audience attention spans.

We'll go with a bold claim. Any movie **over 90 minutes** is considered a long movie.

You need to perform at least 4 major steps. You will receive a full grade for this question ONLY if all 4 steps are present.

1. Formally state your null and alternate hypothesis and state the test statistic that you will be using.
2. Calculate the observed value of your statistic and develop a simulation scheme to simulate your test statistic
3. Visualize the distribution of your test statistic and identify where your observed statistic lies.
4. Use a significance level of 5% and 10% and come to a conclusion for both these significance levels.

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Q8.** Now run the same hypothesis test but this time. Use the adjusted revenue as your variable of observation instead of the custom rating. Run all 4 steps again. Does your conclusion change? What do you think is happening? Why is there a difference in your conclusion as you change the variable you're observing?

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Question 9 (Novel Analysis Part 1):**

Design your own A/B test! Choose one variable whose trends you would like to investigate, and create a new table with that variable's data for the years and states you would like to analyze. Remember to divide the overall table into two groups that you want to compare for the test, and conduct an A/B test investigating the differences between the two groups.

Let's begin the process.

a) Define null and alternate hypotheses.

b) Plot the observed distribution of the variable you will be observing

c) State your test statistic value and what it means in the context of the problem.

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_

**Question 10 (Novel Analysis Part 2):**

Continue the A/B Test from Question 9!


a) Simulate the test statistic under the null hypothesis and visualize the results.

b) Select a significance level for your test and state the conclusion of the hypothesis test.

In [ ]:
#SOLUTION

#### Enter prompt below:

In [ ]:
%%ai openai-chat:gpt-4

#### Explain your answer below:

_type your answer here_